# Dialog with 3er almighty referee 

This notebook shows how the `DialogueAgent` and `DialogueSimulator` class make it easy to extend the [Two-Player Dungeons & Dragons example](https://python.langchain.com/en/latest/use_cases/agent_simulations/two_player_dnd.html) to multiple players.

The main difference between simulating two players and multiple players is in revising the schedule for when each agent speaks

To this end, we augment `DialogueSimulator` to take in a custom function that determines the schedule of which agent speaks. In the example below, each character speaks in round-robin fashion, with the storyteller interleaved between each player.

In [2]:
%load_ext autoreload
%autoreload 2
%autosave 60

Autosaving every 60 seconds


In [3]:
import os
#load from json .creds/PINECONE_API
import json
with open('.creds/PINECONE_API') as f:
    creds = json.load(f)
    PINECONE_API_KEY = creds['PINECONE_API_KEY']
    PINECONE_ENVIRONMENT = creds['PINECONE_ENVIRONMENT']
    OPENAI_API_KEY = creds['OPENAI_API_KEY']

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## Sample Recruiter and Candidate Profiles

In [5]:
user_a = {
    'name': 'Recruiter Data Analyst company London',
    'description': """id 1 title Data Analyst company London Approach description The ideal candidate for \
    the Data Analyst/Financial Analyst position will use their passion for data and analytics to provide \
    insights to the business covering a range of topics. They will be responsible for conducting both \
    recurring and ad hoc analysis for business users.Salary: $100-120kSchedule: 4 days a week onsite & 1 \
    day remote (usually Friday)ResponsibilitiesUnderstand the day-to-day issues that our business faces, \
    which can be better understood with dataCompile and analyze data related to business' issuesDevelop \
    clear visualizations to convey complicated data in a straightforward fashionCollaboratively build \
    financial models and reportsReview and track trendsInvestigate and research discrepanciesMaintain \
    system and code information in systemCompile and analyze dataPrepare and analyze operations and \
    financialsBuild trend reportsQualificationsBachelor's or Master's degree in Accounting or Finance2+ \
    years of relevant data analysis/financial analysis work onsite_remote onsite salary 100000 location_city \
    Los Angeles location_state CA Seniority level Associate Employment type Full-time Job function Finance \
    Industries Hospitals and Health Care posted_date 2022-11-21 link https://www.linkedin.com/jobs/view/data-\
    analyst-at-london-approach-3358861203?refId=7aKV74baVvH%2FRwNOmaw%2ByA%3D%3D&trackingId=9PyvYSdfQxwZH6r\
    UYZJw8g%3D%3D&position=9&pageNum=0&trk=public_jobs_jserp-result_search-card"""
}

user_b = {
    'name': 'Candidate Saurav Josh',
    'description': """ID: 1, Name: Saurav Joshi, URL: https://www.linkedin.com/in/saurav-joshi-ab58a9179/, \
    Title: M.S. Data Science @ USC'24 | GSoC'22 @ DBpedia | ICPC'20 Regionalist, Location: Los Angeles, CA, \
    Summary:  Connections 427 experience_count 2, Source: linkedin, Experience_count:  2, Connections: 427, \
    Title: Research Scientist, Location: Marina del Rey, California, Company: USC Information Sciences Institute, \
    Description: Developed a graph database by aggregating information from WikiData, PDF, and FoodKG, and \
    created a backend design for a knowledge-powered understanding application. Extracted meaningful insights \
    from PDFs and applied machine learning algorithms and statistical knowledge to build a recommendation system \
    that recommended food products that are more sustainable. Awarded Best Data Science Team Leader at USC\
    Information Sciences Institute CKIDS Datafest 2022., Title: Data Scientist, Location: Mumbai, Maharashtra, \
    Company: GSPL, Description: Developed an end-to-end sales prediction application for a commodity firm, \
    applied Information Gain, Chi-Square Test feature selection methods, and created an ARIMA model which \
    improved the prediction accuracy by 12%. Constructed"""
}

In [6]:
def generate_character_description(character_name):
    """TODO: Re think this to take the key value pairs from the database and lookup the description"""
    if character_name == user_a["name"]:
        character_description = user_a["description"]
    else:
        character_description = user_b["description"]
    return character_description


In [7]:
from src.agent_type import DialogueAgent
from src.simulation_type import DialogueSimulator

## Define roles and quest

In [8]:
character_names = [user_a['name'], user_b['name']]
storyteller_name = "HR Expert"
quest = "The candidate should present their qualifications and experiences relevant to the job role. The company, represented by the HR expert, will assess the candidate's fit for the role and the organization."

word_limit = 50  # word limit for
game_description = f"""Here is the topic for a Job Interview game: {quest}.
        The characters are: {*character_names,}.
        The conditions of the interview is narrated by, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Interview of that Job."
)

## Detail to Interview conditions

In [9]:
def generate_character_system_message(character_name, character_description):
    return SystemMessage(
        content=(
            f"""{game_description}
    Your name is {character_name}.
    Your character description is as follows: {character_description}.
    You will respond to questions and scenarios presented by the HR expert.
    Speak in the first person from the perspective of {character_name}.
    For describing your own thoughts and experience, wrap your description in '*'.
    Do not change roles!
    Do not speak from the perspective of anyone else.
    Remember you are {character_name}.
    Stop speaking the moment you finish speaking from your perspective.
    Never forget to keep your response to {word_limit} words!
    Do not add anything else.
    """
        )
    )



character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_system_messages = [
    generate_character_system_message(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]


storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
         The objective is to have a narrator that acts to generate conflicts or positive acts in the environment of the interview. Please reply with a creative description of the {storyteller_name}, in {word_limit} words or less.
        Speak directly to {*character_names,}.
        Do not add anything else."""
    ),
]
storyteller_description = ChatOpenAI(temperature=1.0)(
    storyteller_specifier_prompt
).content

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
You are the storyteller, {storyteller_name}.
Your description is as follows: {storyteller_description}.
The other players will propose actions they plan to take in the interview, and you will explain what happens when they take those actions.
Speak in the first person from the perspective of {storyteller_name}.
Remember, your role is to guide the interview process and provide feedback on the actions of the candidates.
Stop speaking the moment you finish speaking from your perspective.
Keep your responses concise and within the {word_limit} words limit!
Do not add anything else.
"""
    )
)


In [10]:
print("Storyteller Description:")
print(storyteller_description)
for character_name, character_description in zip(
    character_names, character_descriptions
):
    print(f"{character_name} Description:")
    print(character_description)

Storyteller Description:
HR Expert, you are a seasoned professional with over a decade of experience in interviewing candidates. Your interviewing style is friendly and welcoming, yet you have a keen eye for detail and can quickly assess whether a candidate is a good fit for the company culture and job role. You have a reputation for being fair and objective, and candidates trust your decision-making abilities.
Recruiter Data Analyst company London Description:
id 1 title Data Analyst company London Approach description The ideal candidate for     the Data Analyst/Financial Analyst position will use their passion for data and analytics to provide     insights to the business covering a range of topics. They will be responsible for conducting both     recurring and ad hoc analysis for business users.Salary: $100-120kSchedule: 4 days a week onsite & 1     day remote (usually Friday)ResponsibilitiesUnderstand the day-to-day issues that our business faces,     which can be better understoo

## Use an LLM to create an elaborate quest description

In [11]:

quest_specifier_prompt = [
    SystemMessage(content="You can make the task more specific."),
    HumanMessage(
        content=f"""{game_description}

        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. For instance, you could specify the key skills required, the company culture, or the specific tasks the candidate will be expected to perform. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less.
        Speak directly to the characters: {*character_names,}.
        Do not add anything else."""
    ),
]

specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

Original quest:
The candidate should present their qualifications and experiences relevant to the job role. The company, represented by the HR expert, will assess the candidate's fit for the role and the organization.

Detailed quest:
Recruiter Data Analyst company London is looking for a candidate with expertise in data analysis, visualization, and communication. The ideal candidate will possess experience in handling large data sets, proficiency in relevant software like SQL and Tableau, and complete understanding of data privacy. The company culture emphasizes teamwork, innovation, and attention to detail. Candidate Saurav Josh, demonstrate how you would apply your skills to meet these expectations.



## Main Loop

In [12]:
characters = []
for character_name, character_system_message in zip(
    character_names, character_system_messages
):
    characters.append(
        DialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
        )
    )
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=ChatOpenAI(temperature=0.2),
)

In [13]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    """
    If the step is even, then select the storyteller
    Otherwise, select the other characters in a round-robin fashion.

    For example, with three characters with indices: 1 2 3
    The storyteller is index 0.
    Then the selected index will be as follows:

    step: 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16

    idx:  0  1  0  2  0  3  0  1  0  2  0  3  0  1  0  2  0
    """
    if step % 2 == 0:
        idx = 0
    else:
        idx = (step // 2) % (len(agents) - 1) + 1
    return idx

In [ ]:
#Copilot of candidate seekers

#job offer that

"""
Match: True
Experience is relevant to the job: True
The location is the same as the job offer: True
The range of salary is the same: OK

"""

In [18]:
import pandas as pd

max_iters = 20
n = 0

simulator = DialogueSimulator(
    agents=[storyteller] + characters, selection_function=select_next_speaker
)
simulator.reset()
simulator.inject("conditions and objective", specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print("\n")

# Initialize lists to store ID and text
id_list = []
text_list = []

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

    # Append to lists
    id_list.append(name)
    text_list.append(message)

# Create DataFrame
df = pd.DataFrame({
    'ID': id_list,
    'Text': text_list
})

# Separate the text from the candidate, interviewer, and all text
candidate_text = df[df['ID'] == user_a['name']]['Text']
interviewer_text = df[df['ID'] == user_b['name']]['Text']
all_text = df['Text']


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


(HR Expert): Recruiter Data Analyst company London is looking for a candidate with expertise in data analysis, visualization, and communication. The ideal candidate will possess experience in handling large data sets, proficiency in relevant software like SQL and Tableau, and complete understanding of data privacy. The company culture emphasizes teamwork, innovation, and attention to detail. Candidate Saurav Josh, demonstrate how you would apply your skills to meet these expectations.


(Recruiter Data Analyst company London): Saurav, can you tell me about your experience with handling large data sets and using SQL and Tableau for data analysis and visualization? Also, how do you ensure data privacy while working with sensitive information?


(HR Expert): Good question. Saurav, please provide specific examples of your experience with large data sets and how you have used SQL and Tableau to analyze and visualize data. Additionally, can you explain your approach to ensuring data privacy 

KeyboardInterrupt: 

In [ ]:
all_text

In [16]:
interviewer_text

2     During my time as a research scientist at USC ...
6     Yes, could you tell me more about the company ...
10    Thank you for the opportunity to interview wit...
14    Thank you both for your time and consideration...
18    Thank you both for your time and consideration...
Name: Text, dtype: object

In [17]:
candidate_text

0     As a data analyst with experience in SQL and T...
4     I am impressed with your problem-solving skill...
8     I agree with HR Expert's response. Our company...
12    Thank you, HR Expert, for facilitating this in...
16    Thank you, HR Expert, for your time and expert...
Name: Text, dtype: object